## Convective/Stratiform identification for 3D Reflectivity from derived dBZ of WRF Simulations. 

**For [High Resolution WRF Simulations of the Current and Future Climate of North America](https://rda.ucar.edu/datasets/ds612.0/).**

**Hungjui Yu 20210922**

In [2]:
import sys
import time
import datetime as dt
import pytz
# from netCDF4 import Dataset
import numpy as np
import xarray as xr
import pandas as pd
import wrf
from wrf import (getvar, dbz, extract_times, destagger)
# import metpy
# import metpy.calc as mpcalc
# import metpy.units as units
# print(metpy.__version__)

# import dask


ModuleNotFoundError: No module named 'wrf'

**Set input files paths and names:**

In [9]:
def set_input_names(file_date):

    file_path_1 = '/glade/scratch/hungjui/DATA_WRF_CONUS_1_dBZ_v1.0'
    file_path_2 = '/' + wrf_sim_type # '/CTRL3D'
    file_path_3 = '/{}'.format(file_date.strftime('%Y'))

    file_names = dict( dbz = file_path_1 
                           + file_path_2 
                           + file_path_3 
                           + '/wrf3d_d01_' + wrf_sim_type[0:-2] + '_dbz_{}.nc'.format(file_date.strftime('%Y%m%d'))
                     )
    
    return file_names

wrf_sim_type = 'CTRL3D'
file_name_list = set_input_names(dt.datetime(2013,9,13))
print(file_name_list)

{'dbz': '/glade/scratch/hungjui/DATA_WRF_CONUS_1_dBZ_v1.0/CTRL3D/2013/wrf3d_d01_CTRL_dbz_20130913.nc'}


In [12]:
xr.open_dataset(file_name_list['dbz'])
# print(wrf_dbz)

<xarray.Dataset>
Dimensions:  (Time: 8, bottom_top: 50, south_north: 1015, west_east: 1359)
Coordinates:
  * Time     (Time) datetime64[ns] 2013-09-13 ... 2013-09-13T21:00:00
    XLAT     (south_north, west_east) float32 ...
    XLONG    (south_north, west_east) float32 ...
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    Times    (Time) |S19 ...
    XTIME    (Time) float32 ...
    dBZ      (Time, bottom_top, south_north, west_east) float32 ...
Attributes:
    SIMULATION_START_DATE:           2000-10-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        1360
    SOUTH-NORTH_GRID_DIMENSION:      1016
    BOTTOM-TOP_GRID_DIMENSION:       51
    DX:                              4000.0
    DY:                              4000.0
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        3
    DAMPCOEF:                        0.2
    KHDIF:                           0.0
    KVDIF:                           0.0
    MP_PHYSICS:                      28
    RA_LW_PHYSICS:                   4
    RA_SW_PHYSICS:                   4
    SF_SFCLAY_PHYSICS:               11
    SF_SURFACE_PHYSICS:              4
    BL_PBL_PHYSICS:                  1
    CU_PHYSICS:                      0
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      1
    GRID_FDDA:                       2
    GFDDA_INTERVAL_M:                360
    GFDDA_END_H:                     999999
    GRID_SFDDA:                      0
    SGFDDA_INTERVAL_M:               0
    SGFDDA_END_H:                    0
    HYPSOMETRIC_OPT:                 2
    SF_URBAN_PHYSICS:                0
    SHCU_PHYSICS:                    0
    MFSHCONV:                        0
    FEEDBACK:                        0
    SMOOTH_OPTION:                   2
    SWRAD_SCAT:                      1.0
    W_DAMPING:                       1
    MOIST_ADV_OPT:                   1
    SCALAR_ADV_OPT:                  1
    TKE_ADV_OPT:                     1
    DIFF_6TH_OPT:                    0
    DIFF_6TH_FACTOR:                 0.12
    FGDT:                            0.0
    GUV:                             5e-05
    GT:                              5e-05
    GPH:                             5e-05
    IF_RAMPING:                      1
    DTRAMP_MIN:                      60.0
    OBS_NUDGE_OPT:                   0
    BUCKET_MM:                       100.0
    BUCKET_J:                        1000000000.0
    PREC_ACC_DT:                     60.0
    OMLCALL:                         0
    ISFTCFLX:                        0
    ISHALLOW:                        0
    OPT_SFC:                         1
    DVEG:                            4
    OPT_CRS:                         1
    OPT_BTR:                         2
    OPT_RUN:                         1
    OPT_FRZ:                         1
    OPT_INF:                         1
    OPT_RAD:                         3
    OPT_ALB:                         2
    OPT_SNF:                         4
    OPT_TBOT:                        1
    OPT_STC:                         1
    DFI_OPT:                         0
    WEST-EAST_PATCH_START_UNSTAG:    1
    WEST-EAST_PATCH_END_UNSTAG:      1359
    WEST-EAST_PATCH_START_STAG:      1
    WEST-EAST_PATCH_END_STAG:        1360
    SOUTH-NORTH_PATCH_START_UNSTAG:  1
    SOUTH-NORTH_PATCH_END_UNSTAG:    1015
    SOUTH-NORTH_PATCH_START_STAG:    1
    SOUTH-NORTH_PATCH_END_STAG:      1016
    BOTTOM-TOP_PATCH_START_UNSTAG:   1
    BOTTOM-TOP_PATCH_END_UNSTAG:     50
    BOTTOM-TOP_PATCH_START_STAG:     1
    BOTTOM-TOP_PATCH_END_STAG:       51
    GRID_ID:                         1
    PARENT_ID:                       0
    I_PARENT_START:                  1
    J_PARENT_START:                  1
    PARENT_GRID_RATIO:               1
    DT:                              15.0
    CEN_LAT:                         39.700012
    CEN_LON:                    

**Get wrf output variables:**

In [3]:
def get_wrf_vars(file_name, var_name, time_index):

    wrf_file = Dataset(file_name)
    # wrf_var = getvar(wrf_file, wrf_var_to_read, timeidx=time_index_1) # This doesn't work for CONUS run files.
    wrf_var = getvar(wrf_file, var_name, timeidx=time_index, meta=False)
    # wrf_var_time = wrf.extract_times(wrf_file, timeidx=time_index)
    # print(wrf_var_time)
    
    return wrf_var

**Calculation for dBZ:**

In [4]:
def calculate_wrf_dbz(wrf_pres, wrf_temp, wrf_qv, wrf_qr, wrf_qs, wrf_qg):
    
    wrf_dbz = dbz(wrf_pres \
                , wrf_temp \
                , wrf_qv \
                , wrf_qr \
                , wrf_qs \
                , wrf_qg \
                # , use_varint=True \
                , use_liqskin=False \
                , meta=True \
                 )
    
    return wrf_dbz

**Set output file path and name:**

In [5]:
def set_output_name(output_file_datetime):

    # output_path = '/glade/u/home/hungjui/2scratch/DATA_WRF_CONUS_1_dBZ_v1.0'
    
    output_time = pd.to_datetime(output_file_datetime).strftime('%Y%m%d') # If input time type is numpy.datetime64:
    
    # output_name = output_path + '/wrf3d_d01_dbz_{}.nc'.format(file_date_time.strftime('%Y%m%d%H'))
    # output_name = output_path + '/wrf3d_d01_dbz_{}.nc'.format(output_time)
    output_name = '/wrf3d_d01_' + wrf_sim_type[0:-2] + '_dbz_{}.nc'.format(output_time)

    return output_name


### Main Function:

In [6]:
def main_function(file_date_time):
    
    ## Set file datetime:
    # file_date_time = dt.datetime(2013, 9, 13, 0, 0, 0, tzinfo=pytz.utc)
    print('Processing: {}'.format(file_date_time.strftime('%Y%m%d')), end=' ')
    
    ## Set input files paths and names:
    file_name_dict = set_input_names(file_date_time)

    ## Get the 3-hourly time list from P and QRAIN files:
    wrf_3hour_list_1 = wrf.extract_times(Dataset(file_name_dict['P']), timeidx=wrf.ALL_TIMES, meta=False, do_xtime=False)
    wrf_3hour_list_2 = wrf.extract_times(Dataset(file_name_dict['QRAIN']), timeidx=wrf.ALL_TIMES, meta=False, do_xtime=False)

    ## Set wrf variable list for reflectivity retrieval:
    wrf_vars_list = ['P', 'TK', 'QVAPOR', 'QRAIN', 'QSNOW', 'QGRAUP']

    ## Set dBZ data array and append calculated data:
    # wrf_dbz = xr.zeros_like(wrf_dataset_out['P'])
    
    for hi in range(len(wrf_3hour_list_1)):
        
        print(str(hi) + ' | ', end=' ')

        ## Get the index for common time in different files (every 3-hour):
        common_index_2 = np.intersect1d(wrf_3hour_list_1[hi], wrf_3hour_list_2, return_indices=True)[2][0]

        ## Get wrf output variables:
        wrf_vars = {}
        for vname in wrf_vars_list:

            file_name = file_name_dict[vname]

            if ( vname in ['QRAIN', 'QGRAUP'] ):
                wrf_vars['{}'.format(vname)] = get_wrf_vars(file_name, vname, common_index_2)
                # wrf_vars['{}'.format(vname)] = xr.open_dataset(file_name)
            else:
                wrf_vars['{}'.format(vname)] = get_wrf_vars(file_name, vname, hi)
                # wrf_vars['{}'.format(vname)] = xr.open_dataset(file_name)


        ## Calculation for dBZ:
        wrf_dbz_3hr = calculate_wrf_dbz(wrf_vars['P'],
                                        wrf_vars['TK'], 
                                        wrf_vars['QVAPOR'],
                                        wrf_vars['QRAIN'],
                                        wrf_vars['QSNOW'],
                                        wrf_vars['QGRAUP']
                                        ) # .to_dataset()
        
        # wrf_dbz_3hr = wrf_dbz_3hr.expand_dims({'TimeDim': 8})
        # print(wrf_dbz_3hr)
        
        if ( hi == 0 ):
            wrf_dbz = wrf_dbz_3hr
        else:
            wrf_dbz = xr.concat([wrf_dbz, wrf_dbz_3hr], dim='TimeDim')

    # print(wrf_dbz)
            
    ## Set output dataset:
    wrf_dataset_out = xr.open_dataset(file_name_dict['P'])    
      
    ## Add dBZ to output dataset:
    wrf_dataset_out['dBZ'] = (['Time', 'bottom_top', 'south_north', 'west_east'], wrf_dbz)
    #print(wrf_dataset_out)
    
    ## Drop P from the output dataset:
    wrf_dataset_out = wrf_dataset_out.drop_vars('P')
            
    ## Unstagger Z vertical grids:
    # wrf_var_Z_unstag = wrf.destagger(getvar(Dataset(file_name_dict['Z']), 'Z', timeidx=hi, meta=False), 0)

    ## Get AGL:
    # wrf_var_Z_AGL = wrf.g_geoht.get_height_agl(Dataset(file_name_dict['Z']), 'Z', meta=False)

    ## Add Z and dBZ into dataset of P:
    # wrf_dataset_P_Z_dBZ = xr.open_dataset(file_name_dict['P']).isel(Time = hi)
    # wrf_dataset_out = xr.open_dataset(file_name_dict['P'])
    # wrf_dataset_out['dBZ'] = (['bottom_top', 'south_north', 'west_east'], wrf_dbz)
    
    # wrf_dataset_P_Z_dBZ['TK'] = (['bottom_top', 'south_north', 'west_east'], wrf_vars['TK'])
    # wrf_dataset_P_Z_dBZ['Z'] = (['bottom_top', 'south_north', 'west_east'], wrf_var_Z_unstag)
        
    ## Calculate Z using the U.S. standard atmosphere & Hypsometric eqn.:
    # Z_standard = mpcalc.pressure_to_height_std((wrf_dataset_P_Z_dBZ['P'].values) * units.units.Pa)
    # wrf_dataset_P_Z_dBZ['Z_standard'] = (['bottom_top', 'south_north', 'west_east'], Z_standard)

    ## Set coordinates and dimensions:

    ## Set output file path and name:
    output_path_1 = '/glade/u/home/hungjui/2scratch/DATA_WRF_CONUS_1_dBZ_v1.0/' + wrf_sim_type
    # output_path_2 = '/20130913'
    output_path_2 = '/{}'.format(file_date_time.strftime('%Y'))
    output_file_name = set_output_name(file_date_time)
    wrf_dataset_out.to_netcdf(output_path_1 + output_path_2 + output_file_name)
        
    print('Finish this date.')
        

### Main Program:

In [7]:
start = time.time()

## WRF Model Simulation Category:
wrf_sim_type = 'CTRL3D'
# wrf_sim_type = 'PGW3D'

## Loop through a period:
target_date_range = pd.date_range(start='2005-12-27', end='2005-12-31', tz=pytz.utc)

for dayi in target_date_range:
    
    main_function(dayi)
    #main_calc = delayed(main_function)(dayi)
    
# main_calc.compute()
# main_calc.visualize()

end = time.time()
print("RUNTIME：%f SEC" % (end - start))
print("RUNTIME：%f MIN" % ((end - start)/60))
print("RUNTIME：%f HOUR" % ((end - start)/3600))

Processing: 20051227 0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  Finish this date.
Processing: 20051228 0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  Finish this date.
Processing: 20051229 0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  Finish this date.
Processing: 20051230 0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  Finish this date.
Processing: 20051231 0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  Finish this date.
RUNTIME：1141.574093 SEC
RUNTIME：19.026235 MIN
RUNTIME：0.317104 HOUR


In [8]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
